## Gensim LDA 의 (doc, topic) 확률 벡터 calibration 과 perplexity 계산

Gensim 의 LDA 는 각 문서의 토픽 확률 벡터를 추정할 때, 각 토픽에 기본 확률을 부여합니다. 이를 조정하여 sparse 한 토픽 벡터가 되도록 하는 후처리 과정입니다.

Bag of words model 인 `x` 와 미리 학습한 LDA 모델을 로딩합니다. 30,091 개의 문서에서 100 개의 토픽이 학습되었습니다.

In [1]:
import config
import pickle
import numpy as np
import gensim
from gensim import utils
from gensim.matutils import dirichlet_expectation
from lovit_textmining_dataset.navernews_10days import get_bow


x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)
id2word = dict(enumerate(idx_to_vocab))

ldamodel_fname = './2016-10-20-lda.pkl'
with open(ldamodel_fname, 'rb') as f:
    ldamodel = pickle.load(f)

soynlp=0.0.492
added lovit_textmining_dataset


## Perplexity 계산

Perplexity 는 아래의 식을 구현하였습니다.

$\log { \left\{ p(w) \right\}  } =\sum _{ d=1 }^{ D }{ \sum _{ j=1 }^{ V }{ { n }^{ jd }\log { \left[ \sum _{ k=1 }^{ K }{ { \theta  }_{ k }^{ d }{ \phi  }_{ k }^{ j } }  \right]  }  }  }$

$Perplexity(w)=exp\left[ -\frac { log\left\{ p(w) \right\}  }{ \sum _{ d=1 }^{ D }{ \sum _{ j=1 }^{ V }{ { n }^{ jd } }  }  }  \right]$

단, 그 과정에서 (문서, 토픽) 확률과 (토픽, 단어) 확률의 곱에 의해 (문서, 단어) 크기의 numpy.ndarray 가 한 번 계산되는데, 이 크기의 dense matrix 가 한 번에 만들려면 많은 메모리를 이용해야 합니다. 이를 방지하기 위하여 `chunk` 의 개수만큼의 문서만 잘라서 이 과정을 반복하도록 구현하였습니다.

또한 (문서, 단어) 벡터 행렬의 값을 로그로 변환하는 과정에서 log (0) -inf 로 계산되는 것을 방지하기 위하여 그 곱이 0 인 값은 1 로 변환하는 부분을 추가하였습니다.

In [2]:
import numpy as np
import math
from sklearn.utils.extmath import safe_sparse_dot
from scipy.sparse import csr_matrix

def compute_perplexity(bow, topic_term_prob, doc_topic_prob, chunk=1000):
    n_docs = bow.shape[0]
    log_perp = 0
    for c in range(math.ceil(n_docs / chunk)):
        b = int(c * chunk)
        e = min(n_docs, int((c+1) * chunk))
        doc_term_prod = doc_topic_prob[b:e].dot(topic_term_prob)
        doc_term_prod[np.where(doc_term_prod == 0)] = 1
        doc_term_prod = -1 * np.log(doc_term_prod)
        doc_term_prod = np.nan_to_num(doc_term_prod)
        log_perp += bow[b:e].multiply(doc_term_prod).sum()
    log_perp /= bow.sum()
    return log_perp

아래는 학습된 Gensim LDA 에서 (문서, 토픽) 확률 행렬과 (토픽, 단어) 확률 행렬을 가져오는 함수입니다.

In [3]:
def get_parameters(lda, corpus):
    topic_term_prob = _get_topic_term_prob(lda)
    doc_topic_freq = _get_doc_topic_freq(lda, corpus)

    doc_topic_prob = doc_topic_freq / doc_topic_freq.sum(axis=1)[:, None]
    topic_prob = doc_topic_freq.sum(axis=0) / doc_topic_freq.sum()

    return topic_term_prob, doc_topic_prob, topic_prob

def _get_topic_term_prob(lda):
    topic_term_freq = lda.state.get_lambda()
    topic_term_prob = topic_term_freq / topic_term_freq.sum(axis=1)[:, None]
    return topic_term_prob

def _get_doc_topic_freq(lda, corpus):
    doc_topic_freq, _ = lda.inference(corpus)
    return doc_topic_freq

topic_term_prob, doc_topic_prob, topic_prob = get_parameters(ldamodel, corpus)

학습된 LDA 모델에 대한 perplexity 를 계산합니다. 아래의 값은 log 를 씌운 값입니다.

In [4]:
compute_perplexity(x, topic_term_prob, doc_topic_prob)

7.36096462712405

그런데 1 번 문서의 토픽 벡터를 살펴보면 대부분의 토픽에 0.2 % 의 확률이 부여되어 있습니다. 사실 이 문서를 표현하는 토픽은 0.2 보다 큰 세 개의 토픽인데, 0.2 % 의 97 개 토픽에 의하여 약 20 % 의 확률이 쓰이질 못하고 있습니다. 이 값을 0 으로 변환하여 sparse 한 토픽 벡터를 만들어 봅니다.

In [5]:
doc_topic_prob[1]

array([0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.27117783, 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002     ,
       0.002     , 0.20199998, 0.002     , 0.002     , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.3328221 , 0.002     ,
       0.002     , 0.002     , 0.002     , 0.002     , 0.002  

`calibrate_doc_topic_prob` 함수는 각 문서의 토픽 벡터에서 최소값을 찾은 뒤, 벡터에서 이 값을 빼고 벡터의 L1 norm 이 1 이 되도록 조절하는 함수입니다.

In [6]:
from sklearn.preprocessing import normalize

def calibrate_doc_topic_prob(doc_topic_prob, min_prob=None, uniform_as_zero=False):
    n_docs, n_topics = doc_topic_prob.shape
    if min_prob is None:
        min_prob = doc_topic_prob.min(axis=1)
    if n_docs != min_prob.shape[0]:
        raise ValueError('min_prob length must be same with num of docs')
    shift = np.repeat(min_prob, n_topics, axis=0).reshape(-1, n_topics)
    calibrated_doc_topic_prob = doc_topic_prob - shift
    calibrated_doc_topic_prob = normalize(calibrated_doc_topic_prob, norm='l1')
    if uniform_as_zero:
        calibrated_doc_topic_prob = probablity_normalization(calibrated_doc_topic_prob)
    return calibrated_doc_topic_prob

def probablity_normalization(prob):
    row_sum = prob.sum(axis=1)
    base = 1 / prob.shape[1]
    prob[np.where(row_sum == 0)[0]] = base
    return prob

calibrated_doc_topic_prob = calibrate_doc_topic_prob(doc_topic_prob)

함수 적용 결과 세 개의 토픽의 확률값은 이전보다 커졌고, 나머지 97 개의 토픽의 확률값은 0 이 되었습니다.

In [7]:
calibrated_doc_topic_prob[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.33647233, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.25      , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.4135277 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

이전의 log perplexity 는 7.36096462712405 이었는데, 그 때보다는 log perplexity 가 조금 줄어들었음을 알 수 있습니다. 이는 각 확률에 부여되는 기본값에 의하여 LDA 의 설명력이 약해졌다는 의미입니다. Sparse topic vector 로 변환함으로써 조금 더 설명력이 좋은 LDA 모델이 되었습니다.

In [8]:
compute_perplexity(x, topic_term_prob, calibrated_doc_topic_prob)

7.355193965112524

## Merging topic

때로는 후처리 과정에서 몇 개의 토픽을 하나로 병합하고 싶기도 합니다. `merge_topic` 함수는 이를 위한 함수로, (토픽, 단어) 행렬, (문서, 토픽) 행렬, 문서의 길이 벡터, 그리고 병합할 토픽의 아이디를 입력하면 이를 병합하는 함수입니다. 병합되지 않는 토픽을 맨 앞에, 병합하는 토픽이 마지막의 아이디를 가지도록 만들었습니다.

각 문서마다 길이가 다르기 때문에 토픽을 병합할 때는 토픽 별 가중치가 달라야 합니다. (문서, 토픽) 확률 행렬과 문서의 길이 벡터를 이용하여 (문서, 토픽) 빈도 행렬을 만든 뒤, 각 토픽의 확률을 계산합니다. 이 값을 가중치로 이용하여 토픽을 병합합니다.

In [9]:
from sklearn.preprocessing import normalize

def merge_topic(topic_term_prob, doc_topic_prob, doc_lengths, merge_topic_idxs):
    pos_idx = np.asarray(merge_topic_idxs)
    neg_idx = np.asarray(
        [i for i in range(topic_term_prob.shape[0])
         if not (i in merge_topic_idxs)]
    )

    doc_topic_freq = np.diag(doc_lengths).dot(doc_topic_prob)
    topic_freq = np.asarray(doc_topic_freq.sum(axis=0)).reshape(-1)
    topic_term_freq = np.diag(topic_freq).dot(topic_term_prob)

    doc_topic_freq_pos = np.asarray(doc_topic_freq[:,pos_idx].sum(axis=1)).reshape(-1,1)
    doc_topic_freq_neg = doc_topic_freq[:,neg_idx]
    doc_topic_prob_ = np.hstack([doc_topic_freq_pos, doc_topic_freq_neg])
    topic_prob_ = normalize(np.asarray(doc_topic_prob_.sum(axis=0)).reshape(1,-1)).reshape(-1)
    doc_topic_prob_ = normalize(doc_topic_prob_, norm='l1')

    topic_term_pos = np.asarray(topic_term_freq[pos_idx].sum(axis=0)).reshape(1,-1)
    topic_term_neg = topic_term_freq[neg_idx]
    topic_term_prob_ = normalize(np.vstack([topic_term_pos, topic_term_neg]), norm='l1')

    return doc_topic_prob_, topic_term_prob_, topic_prob_

def sum_bow(bow):
    doc_lengths = np.asarray(bow.sum(axis=1)).reshape(-1)
    term_frequency = np.asarray(bow.sum(axis=0)).reshape(-1)
    return doc_lengths, term_frequency


임의로 1, 3, 4 번의 토픽을 하나로 병합하였습니다.

In [10]:
doc_lengths, term_frequency = sum_bow(x)
doc_topic_prob_, topic_term_prob_, topic_prob_ = merge_topic(topic_term_prob, doc_topic_prob, doc_lengths, [1,3,4])

doc_topic_prob_ = probablity_normalization(doc_topic_prob_)
topic_term_prob_ = probablity_normalization(topic_term_prob_)

그 결과 100 개에서 98 개로 토픽의 개수가 줄어든 (문서, 토픽) 행렬이 만들어집니다.

In [11]:
doc_topic_prob_.shape

(30091, 98)

이 역시 LDAvis 를 이용하여 시각화 할 수 있습니다. 결과를 살펴보면 토픽의 개수가 98 개로 줄어들었습니다.

In [12]:
from pyLDAvis import prepare, show, save_html

prepared_data = prepare(
    topic_term_prob_,
    doc_topic_prob_,
    doc_lengths,
    idx_to_vocab,
    term_frequency,
    mds = 'tsne',
    plot_opts = {'xlab': 't-SNE1', 'ylab': 't-SNE2'},
    R = 30 # num of displayed terms
)

save_html(prepared_data, './2016-10-20-pyldavis_t98.html')

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
